In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


from sklearn.preprocessing import StandardScaler

import pylab as plt
%matplotlib inline

from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import warnings
warnings.filterwarnings('ignore') 
plt.rcParams['figure.figsize'] = (10, 8)
# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('epldata_final.csv')

In [ ]:
df.head()

In [ ]:
def category(x):
    if 0 <= x < 15: return 1
    elif 15 <= x < 50: return 2
    else: return 3

In [ ]:
# делаем задачу классификации
df['market_value'] =  df['market_value'].apply(category)

In [ ]:
df['region'].fillna((df['region'].median()), inplace=True)
#df.isnull().any()

In [ ]:
df = df.drop(['age_cat'], axis=1)

In [ ]:
def create_df(data_fram, feature_list):
    out = data_fram
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

df1 = df
features = ['club', 'position', 'nationality', 'position_cat', 'region']
for feature in features:
    df1[feature] = df1[feature].astype('object')
df1 = create_df(df1, features)
df1.head()

In [ ]:
y = df1['market_value']
df1 = df1.drop(['name', 'market_value', 'fpl_sel'], axis=1)

In [ ]:
from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(df1)
# standardize the data attributes
standardized_X = preprocessing.scale(df1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df1, y, test_size = 0.3, random_state=17)

In [ ]:
dict_predictions={}

In [ ]:
#adaboost
abc = AdaBoostClassifier(learning_rate=0.05, algorithm="SAMME")
abc.fit(X_train, y_train)

In [ ]:
abc_pred = abc.predict(X_valid)
dict_predictions['AdaBoostClassifier'] = abc_pred

In [ ]:
#xgb
xgb_model = xgb.XGBClassifier(
                    max_depth=7,
                    learning_rate=0.1,
                    n_estimators=100,
                    loss_function='MultiClass'
)

xgb_model.fit(X_train, y_train)

In [ ]:
xgb_pred = xgb_model.predict(X_valid)
dict_predictions['XGBClassifier'] = xgb_pred

In [ ]:
#lgbm
bst = lgb.LGBMClassifier(max_depth=7, learning_rate=0.1, loss_function='MultiClass')
bst.fit(X_train, y_train)

In [ ]:
lgb_pred = bst.predict(X_valid)
dict_predictions['Lightgbm'] = lgb_pred

In [ ]:
#catboost
cbc = cb.CatBoostClassifier(depth=7, learning_rate=0.1, n_estimators=100, loss_function='MultiClass')
cbc.fit(X_train, y_train)

In [ ]:
cbc_pred = cbc.predict(X_valid)
dict_predictions['CatBoost'] = cbc_pred

In [ ]:
for i in dict_predictions:
    print(i, 'accuracy #', accuracy_score(y_valid, dict_predictions[i]))